In [ ]:
import io
import requests
from zipfile import ZipFile
EXCEL_PATH = 'online_retail_II.xlsx' # put the Excel file in the same folder or change this path
DOWNLOAD_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online%20retail%20ii.xlsx'
OUTPUT_CSV = 'online_retail_cleaned.csv'
def load_excel(path=EXCEL_PATH, download_url=None):
    """Load Excel workbook(s) from a local file, a zip archive containing an Excel file,
    or from a download URL. Returns a dict of DataFrames (sheet_name -> DataFrame).

    This function will:
    - If `path` exists and is a zip, open the zip and read the first Excel file inside.
    - If `path` exists and is an .xlsx/.xls, read it with engine='openpyxl' where possible.
    - If `download_url` is provided, download bytes and detect whether it's a zip or Excel file.
    """
    from zipfile import ZipFile

    def _read_bytes_as_excel(bts):
    # prefer openpyxl for xlsx; fall back to default on error
    try:
            return pd.read_excel(io.BytesIO(bts), sheet_name=None, engine='openpyxl')
    except Exception:
            return pd.read_excel(io.BytesIO(bts), sheet_name=None)

    # Local path handling
    if os.path.exists(path):
    print(f"Loading local Excel file or archive: {path}")
    # If zip archive, open and find the first Excel file inside
    if str(path).lower().endswith('.zip'):
            with ZipFile(path) as z:
                xlsx_files = [n for n in z.namelist() if n.lower().endswith(('.xlsx', '.xlsm', '.xls'))]
                if not xlsx_files:
                    raise RuntimeError(f"No Excel file found inside zip archive: {path}")
                fname = xlsx_files[0]
                print(f"Found Excel file inside zip: {fname}")
                with z.open(fname) as f:
                    content = f.read()
                return _read_bytes_as_excel(content)
    else:
            # Regular Excel file
            try:
                return pd.read_excel(path, sheet_name=None, engine='openpyxl')
            except Exception:
                return pd.read_excel(path, sheet_name=None)

    # Download URL path
    if download_url:
    try:
            print(f"Attempting to download from: {download_url}")
            import requests
            r = requests.get(download_url)
            r.raise_for_status()
            content = r.content
            # detect zip by PK signature
            if content[:4] == b'PK\x03\x04':
                with ZipFile(io.BytesIO(content)) as z:
                    xlsx_files = [n for n in z.namelist() if n.lower().endswith(('.xlsx', '.xlsm', '.xls'))]
                    if not xlsx_files:
                        raise RuntimeError("No Excel file found inside downloaded zip archive")
                    fname = xlsx_files[0]
                    print(f"Found Excel file inside downloaded zip: {fname}")
                    with z.open(fname) as f:
                        content = f.read()
                    return _read_bytes_as_excel(content)
            else:
                return _read_bytes_as_excel(content)
    except Exception as e:
            raise RuntimeError(f"Failed to download or read Excel file: {e}")

    raise FileNotFoundError(f"Excel file not found at {path}. Set EXCEL_PATH or provide DOWNLOAD_URL.")
{
    "cells": [
        {
            "cell_type": "markdown",
            "id": "#VSC-ce602762",
            "metadata": {
                "language": "markdown"
            },
            "source": [
                "",
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-c1f2f280",
            "metadata": {
                "language": "python"
            },
            "source": [
                "!pip install openpyxl",
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-cc6fc4a2",
            "metadata": {
                "language": "python"
            },
            "source": [
                "import os",
                "import io",
                "import zipfile",
                "from pathlib import Path",
                "from datetime import timedelta",
                "",
                "",
                "import pandas as pd",
                "import numpy as np",
                "import matplotlib.pyplot as plt",
                "import seaborn as sns",
                "",
                "# Machine learning imports",
                "from sklearn.ensemble import RandomForestClassifier",
                "from sklearn.model_selection import train_test_split",
                "from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report",
                "import joblib"
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-cf208bb7",
            "metadata": {
                "language": "python"
            },
            "source": [
                "EXCEL_PATH = 'online_retail_II.xlsx' # put the Excel file in the same folder or change this path",
                "DOWNLOAD_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online%20retail%20ii.xlsx'",
                "OUTPUT_CSV = 'online_retail_cleaned.csv'"
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-7da2b61f",
            "metadata": {
                "language": "python"
            },
            "source": [
                "",
                "",
                "def load_excel(path=EXCEL_PATH, download_url=None):",
                "    if os.path.exists(path):",
                "        print(f\"Loading local Excel file: {path}\")",
                "        xls = pd.read_excel(path, sheet_name=None)",
                "        return xls",
                "    elif download_url:",
                "        try:",
                "            print(f\"Attempting to download from: {download_url}\")",
                "            import requests",
                "            r = requests.get(download_url)",
                "            r.raise_for_status()",
                "            # read Excel file from bytes",
                "            xls = pd.read_excel(io.BytesIO(r.content), sheet_name=None)",
                "            return xls",
                "        except Exception as e:",
                "            raise RuntimeError(f\"Failed to download or read Excel file: {e}\")",
                "    else:",
                "        raise FileNotFoundError(f\"Excel file not found at {path}. Set EXCEL_PATH or provide DOWNLOAD_URL.\")",
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-03ff5f30",
            "metadata": {
                "language": "python"
            },
            "source": [
                "# Load data",
                "sheets = load_excel(EXCEL_PATH, download_url=DOWNLOAD_URL)",
                "print('Sheets found:', list(sheets.keys()))"
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-2b776987",
            "metadata": {
                "language": "python"
            },
            "source": [
                "df_list = []",
                "for name, df in sheets.items():",
                "    df['__source_sheet'] = name",
                "    df_list.append(df)",
                "",
                "",
                "data = pd.concat(df_list, ignore_index=True)"
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-93d7e965",
            "metadata": {
                "language": "python"
            },
            "source": [
                "data.head()"
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-f7fe116f",
            "metadata": {
                "language": "python"
            },
            "source": [
                "data.info()",
                ""
            ]
        },
        {
            "cell_type": "markdown",
            "id": "#VSC-60a883d7",
            "metadata": {
                "language": "markdown"
            },
            "source": [
                "### First observations and Data Cleaning Steps",
                "",
                "- 'InvoiceDate' багана нь огноо/цагийн мэдээллийг агуулж байгаа ч одоогоор 'object' (стринг) төрөлтэй байна — энэ нь datetime үйлдлүүдийг хийхийн тулд хөрвүүлэлт шаардлагатай. InvoiceDate_DT баганыг үүсгэн, 'InvoiceDate'-г datetime болгож хөрвүүлнэ.",
                "- Ихэнх баганууд нь 1067371 мөртэй. Гэхдээ 'Description' багана нь 1062989 мөр, 'Customer ID' нь 824364 мөртэй байна — зарим өгөгдөл алга байна. 'Description'-ийг их ашиглахгүй ч 'Customer ID' нь зайлшгүй хэрэгтэй тул эдгээр мөрүүдийг устгана.",
                "- 'Customer ID' одоогоор 'float64' төрөлтэй байна, 'int64' бол илүү тохиромжтой — тухайн баганыг бүхэл тоон төрлөөр хөрвүүлэх болно.",
                "- Бүтээгдэхүүний үнэ ба тоо (quantity) байгаа тул дансны нийт үнийг илрүүлэх зорилгоор тусад нь 'TotalValue' баганыг үүсгэнэ.",
                "- Эцэст нь өгөгдлийг огноогоор (chronologically) эрэмбэлнэ.",
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-8ef23553",
            "metadata": {
                "language": "python"
            },
            "source": [
                "# Preprocessing steps:",
                "# 1. Convert InvoiceDate to datetime in a separate column InvoiceDate_DT",
                "# 2. Drop rows with missing Customer ID",
                "# 3. Convert Customer ID to integer",
                "# 4. Compute TotalValue = Quantity * Price (or UnitPrice)",
                "# 5. Sort by InvoiceDate_DT",
                "",
                "# make a copy to avoid SettingWithCopyWarning if needed",
                "data = data.copy()",
                "",
                "# Convert InvoiceDate to datetime, keep original as object and store parsed in InvoiceDate_DT",
                "if 'InvoiceDate' in data.columns:",
                "    data['InvoiceDate_DT'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')",
                "else:",
                "    print('Warning: InvoiceDate column not found')",
                "",
                "# Drop rows without Customer ID since it's required for analysis",
                "if 'Customer ID' in data.columns:",
                "    before = len(data)",
                "    data = data[~data['Customer ID'].isna()].copy()",
                "    after = len(data)",
                "    print(f'Dropped {before - after} rows with missing Customer ID')",
                "    # Convert Customer ID to integer where possible",
                "    # If Customer ID is float (e.g., 12345.0) converting via Int64 nullable dtype is safer",
                "    try:",
                "        data['Customer ID'] = data['Customer ID'].astype('Int64')",
                "    except Exception:",
                "        # fallback: convert to int after filling/rounding",
                "        data['Customer ID'] = data['Customer ID'].astype(float).round().astype('Int64')",
                "else:",
                "    print('Warning: Customer ID column not found')",
                "",
                "# Compute TotalValue. Possible price column names: 'Price', 'UnitPrice', 'Unit Price', 'PricePerUnit' - try common ones",
                "price_cols = [c for c in data.columns if c.lower() in ('price', 'unitprice', 'unit price', 'unit_price', 'priceperunit')]",
                "qty_col = None",
                "for name in data.columns:",
                "    if name.lower() in ('quantity', 'qty', 'units'):",
                "        qty_col = name",
                "        break",
                "",
                "if price_cols and qty_col:",
                "    price_col = price_cols[0]",
                "    data['TotalValue'] = data[qty_col].fillna(0) * data[price_col].fillna(0)",
                "    print(f'Computed TotalValue using {qty_col} * {price_col}')",
                "else:",
                "    # If price or qty not found, attempt common column names used in this dataset: 'Price' often is 'Price' and quantity is 'Quantity'",
                "    if 'Quantity' in data.columns and 'Price' in data.columns:",
                "        data['TotalValue'] = data['Quantity'].fillna(0) * data['Price'].fillna(0)",
                "        print('Computed TotalValue using Quantity * Price')",
                "    else:",
                "        print('Could not compute TotalValue: Quantity or Price column not found')",
                "",
                "# Finally sort by InvoiceDate_DT if present",
                "if 'InvoiceDate_DT' in data.columns:",
                "    data = data.sort_values('InvoiceDate_DT').reset_index(drop=True)",
                "    print('Data sorted by InvoiceDate_DT')",
                "",
                "# Show resulting info and head for verification",
                "data.info()",
                "data.head()"
            ]
        },
        {
            "cell_type": "markdown",
            "id": "#VSC-7f18a9c8",
            "metadata": {
                "language": "markdown"
            },
            "source": [
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-2cc84bd5",
            "metadata": {
                "language": "python"
            },
            "source": [
                ""
            ]
        },
        {
            "cell_type": "code",
            "id": "#VSC-e3a050bd",
            "metadata": {
                "language": "python"
            },
            "source": [
                ""
            ]
        }
    ]
}

In [41]:
!pip install openpyxl


In [42]:
import os
import io
import zipfile
from pathlib import Path
from datetime import timedelta


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import joblib

In [43]:
EXCEL_PATH = 'online_retail_II.xlsx' # put the Excel file in the same folder or change this path
DOWNLOAD_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online%20retail%20ii.xlsx'
OUTPUT_CSV = 'online_retail_cleaned.csv'

In [44]:


def load_excel(path=EXCEL_PATH, download_url=None):
    if os.path.exists(path):
        print(f"Loading local Excel file: {path}")
        xls = pd.read_excel(path, sheet_name=None)
        return xls
    elif download_url:
        try:
            print(f"Attempting to download from: {download_url}")
            import requests
            r = requests.get(download_url)
            r.raise_for_status()
            # read Excel file from bytes
            xls = pd.read_excel(io.BytesIO(r.content), sheet_name=None)
            return xls
        except Exception as e:
            raise RuntimeError(f"Failed to download or read Excel file: {e}")
    else:
        raise FileNotFoundError(f"Excel file not found at {path}. Set EXCEL_PATH or provide DOWNLOAD_URL.")


In [45]:
# Load data
sheets = load_excel(EXCEL_PATH, download_url=DOWNLOAD_URL)
print('Sheets found:', list(sheets.keys()))

Loading local Excel file: online_retail_II.xlsx
Sheets found: ['Year 2009-2010', 'Year 2010-2011']
Sheets found: ['Year 2009-2010', 'Year 2010-2011']


In [46]:
df_list = []
for name, df in sheets.items():
    df['__source_sheet'] = name
    df_list.append(df)


data = pd.concat(df_list, ignore_index=True)

In [47]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,__source_sheet
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,Year 2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Year 2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Year 2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,Year 2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,Year 2009-2010


In [48]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Invoice         1067371 non-null  object        
 1   StockCode       1067371 non-null  object        
 2   Description     1062989 non-null  object        
 3   Quantity        1067371 non-null  int64         
 4   InvoiceDate     1067371 non-null  datetime64[ns]
 5   Price           1067371 non-null  float64       
 6   Customer ID     824364 non-null   float64       
 7   Country         1067371 non-null  object        
 8   __source_sheet  1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 73.3+ MB


### First observations and Data Cleaning Steps

- 'InvoiceDate' багана нь огноо/цагийн мэдээллийг агуулж байгаа ч одоогоор 'object' (стринг) төрөлтэй байна — энэ нь datetime үйлдлүүдийг хийхийн тулд хөрвүүлэлт шаардлагатай. InvoiceDate_DT баганыг үүсгэн, 'InvoiceDate'-г datetime болгож хөрвүүлнэ.
- Ихэнх баганууд нь 1067371 мөртэй. Гэхдээ 'Description' багана нь 1062989 мөр, 'Customer ID' нь 824364 мөртэй байна — зарим өгөгдөл алга байна. 'Description'-ийг их ашиглахгүй ч 'Customer ID' нь зайлшгүй хэрэгтэй тул эдгээр мөрүүдийг устгана.
- 'Customer ID' одоогоор 'float64' төрөлтэй байна, 'int64' бол илүү тохиромжтой — тухайн баганыг бүхэл тоон төрлөөр хөрвүүлэх болно.
- Бүтээгдэхүүний үнэ ба тоо (quantity) байгаа тул дансны нийт үнийг илрүүлэх зорилгоор тусад нь 'TotalValue' баганыг үүсгэнэ.
- Эцэст нь өгөгдлийг огноогоор (chronologically) эрэмбэлнэ.


In [49]:
# Preprocessing steps:
# 1. Convert InvoiceDate to datetime in a separate column InvoiceDate_DT
# 2. Drop rows with missing Customer ID
# 3. Convert Customer ID to integer
# 4. Compute TotalValue = Quantity * Price (or UnitPrice)
# 5. Sort by InvoiceDate_DT

# make a copy to avoid SettingWithCopyWarning if needed
data = data.copy()

# Convert InvoiceDate to datetime, keep original as object and store parsed in InvoiceDate_DT
if 'InvoiceDate' in data.columns:
    data['InvoiceDate_DT'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
else:
    print('Warning: InvoiceDate column not found')

# Drop rows without Customer ID since it's required for analysis
if 'Customer ID' in data.columns:
    before = len(data)
    data = data[~data['Customer ID'].isna()].copy()
    after = len(data)
    print(f'Dropped {before - after} rows with missing Customer ID')
    # Convert Customer ID to integer where possible
    # If Customer ID is float (e.g., 12345.0) converting via Int64 nullable dtype is safer
    try:
        data['Customer ID'] = data['Customer ID'].astype('Int64')
    except Exception:
        # fallback: convert to int after filling/rounding
        data['Customer ID'] = data['Customer ID'].astype(float).round().astype('Int64')
else:
    print('Warning: Customer ID column not found')

# Compute TotalValue. Possible price column names: 'Price', 'UnitPrice', 'Unit Price', 'PricePerUnit' - try common ones
price_cols = [c for c in data.columns if c.lower() in ('price', 'unitprice', 'unit price', 'unit_price', 'priceperunit')]
qty_col = None
for name in data.columns:
    if name.lower() in ('quantity', 'qty', 'units'):
        qty_col = name
        break

if price_cols and qty_col:
    price_col = price_cols[0]
    data['TotalValue'] = data[qty_col].fillna(0) * data[price_col].fillna(0)
    print(f'Computed TotalValue using {qty_col} * {price_col}')
else:
    # If price or qty not found, attempt common column names used in this dataset: 'Price' often is 'Price' and quantity is 'Quantity'
    if 'Quantity' in data.columns and 'Price' in data.columns:
        data['TotalValue'] = data['Quantity'].fillna(0) * data['Price'].fillna(0)
        print('Computed TotalValue using Quantity * Price')
    else:
        print('Could not compute TotalValue: Quantity or Price column not found')

# Finally sort by InvoiceDate_DT if present
if 'InvoiceDate_DT' in data.columns:
    data = data.sort_values('InvoiceDate_DT').reset_index(drop=True)
    print('Data sorted by InvoiceDate_DT')

# Show resulting info and head for verification
data.info()
data.head()

Dropped 243007 rows with missing Customer ID
Computed TotalValue using Quantity * Price
Data sorted by InvoiceDate_DT
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824364 entries, 0 to 824363
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Invoice         824364 non-null  object        
 1   StockCode       824364 non-null  object        
 2   Description     824364 non-null  object        
 3   Quantity        824364 non-null  int64         
 4   InvoiceDate     824364 non-null  datetime64[ns]
 5   Price           824364 non-null  float64       
 6   Customer ID     824364 non-null  Int64         
 7   Country         824364 non-null  object        
 8   __source_sheet  824364 non-null  object        
 9   InvoiceDate_DT  824364 non-null  datetime64[ns]
 10  TotalValue      824364 non-null  float64       
dtypes: Int64(1), datetime64[ns](2), float64(2), int64(1), object(5)
memory usage:

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,__source_sheet,InvoiceDate_DT,TotalValue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,Year 2009-2010,2009-12-01 07:45:00,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,Year 2009-2010,2009-12-01 07:45:00,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,Year 2009-2010,2009-12-01 07:45:00,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,Year 2009-2010,2009-12-01 07:45:00,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,Year 2009-2010,2009-12-01 07:45:00,30.0
